In [5]:
! pip install praw

     |████████████████████████████████| 163kB 9.8MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 


In [8]:
from flask import Flask, render_template
import flask
import pickle
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import pandas as pd
import numpy as np
import praw
from textblob import TextBlob
import sys
import xgboost

app = Flask(__name__)
def Subjectivity(text):
    return TextBlob(text).sentiment.subjectivity
def Polarity(text):
    return TextBlob(text).sentiment.polarity
def word_count(text):
    wordList = re.sub("[^\w]", " ", text).split()
    return len(wordList)
def clean_message(text):
    text = re.sub(r'[^\w\s]', '', text)
    l_text = " ".join(word for word in text.lower().split() if word not in ENGLISH_STOP_WORDS)
    return l_text
      
# for model Prediction
xgb_b=xgboost.Booster()
xgb_b.load_model('XGBoostRegressor.booster')


# To get information for the Reddit url
def extract_data(url):
    data = {}
    reddit = praw.Reddit(client_id='WUTH6H3Cx7KW4w',
                         client_secret='hBOWXZ37WOOY9M9oT-SD-2H7ql_7HQ',
                         user_agent='user_agent')
    sub_data = reddit.submission(url=str(url))
    data['body'] = [str(sub_data.title)]
    data['downs'] = sub_data.downs
    data['upvote_ratio']=sub_data.upvote_ratio
    data['gilded'] = [sub_data.gilded]
    data['word_count'] = word_count(sub_data.title)
    data['over_18'] = [sub_data.over_18]
    data['number_of_Comments'] = [sub_data.num_comments]
    data['Subjectivity'] = Subjectivity(sub_data.title)
    data['Polarity'] = Polarity(sub_data.title)
    scores = senti.polarity_scores(sub_data.title)
    data['Compound'] = scores['compound']
    data['neg'] = scores['neg']
    data['neu'] = scores['neu']
    data['pos'] = scores['pos']
    df = pd.DataFrame(data)
        
    return df

@app.route('/')
def home():
    return render_template('Index.html')

[16:44:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
